In [1]:
import os
import sys
os.chdir("../")
sys.path.append("../")

import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
# from thop import profile
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.TwoTowerBaseModel import TwoTowerBaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.TESRec import TESRec
from models.XFormer import XFormer
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [3]:
news = pickle.load(open("/data/v-pezhang/Code/Document-Reduction/Code/data/cache/MIND/news/bert/MINDlarge_dev/news.pkl", "rb"))

In [6]:
news['encoded_news'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))
m4 = AutoModel.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', cache_dir=config.path + "bert_cache/")
# t4 = AutoTokenizer.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")

In [16]:
m4.config.axial_pos_shape = (40, 64)

In [20]:
pat = re.search('/(MIND(\w*?)_(.*)/)', "/MINDlarge_dev/")

In [24]:
pat.group(3)

'dev'

In [29]:
class A():
    def __init__(self) -> None:
        self.a()
    
    def a(self):
        print("fuck")

class B(A):
    def __init__(self):
        super().__init__()
    # def a(self):
    #     print("shit")

c = B()

fuck


In [ ]:
t4.pad_token = t4.eos_token
x = t4("He loves embedings", return_tensors="pt", padding="max_length", max_length=20, add_special_tokens=True)

In [ ]:
output = m4(**x)

In [ ]:
output[0].shape

In [ ]:
t4.convert_ids_to_tokens(2)

In [ ]:
m4.config.hidden_dropout_prob

In [ ]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'longformer'
# config.seed = None
config.mode = "recall"
config.recall_type = "s"
config.scale = "large"
config.case = False

manager = Manager(config)

loaders = manager.prepare()
X1 = list(loaders[0])
# X2 = list(loaders[1])
x1 = X1[0]
# x2 = X2[0]

In [ ]:
embedding = BERT_Embedding(manager)

encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)
# model = XFormer(manager).to(manager.device)

In [ ]:
manager.load(model, 230000)

In [ ]:
a,b,c,d,e = model.encode_user(x1)

In [ ]:
np.asarray(t.convert_ids_to_tokens(x1['his_encoded_index'][0,0])[1:])[[86,2,93]]

In [ ]:
e[0,3,35], e.shape

In [ ]:
t.convert_ids_to_tokens(x1['his_encoded_index'][0,3,[36]])

In [ ]:
print(t.convert_ids_to_tokens(x1['his_encoded_index'][0,3]))

In [ ]:
e[0,3], e.shape, e.topk(dim=-1, k=3)

In [ ]:
recall = 0
count = 0
for x in tqdm(loaders[0], ncols=120, leave=True):
    kid = model.encode_user(x)[1]
    ps_terms = x['his_encoded_index'][:, :, 1:].to(manager.device).gather(index=kid, dim=-1).view(-1).tolist()
    

In [ ]:
embedding = 32*50*2*100*768
ext_encoding = 32*50*(2*3*384*100 + 4*100*384) + 32*8*2*384*384*50
reduction = (2*100*768 + 100*math.log2(100))*32*50*2+32*50*3
bert_embedding = 32*150*768*2
bert_project = 32*150*768*2*3
bert_attn = 32*150*12*64*64*2 + 32*150*12*12*64*2
bert_intm = 32*150*768*768*2 + 32*150*768*2 + 32*150*768*3072*4
bert_pool = 32*150*768*4
bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

total_isrec = embedding + ext_encoding + reduction + bert

embedding = 32*50*2*100*768 + 32*50*100*768*2
bert_project = 32*5000*768*2*3
bert_attn = 32*5000*12*64*64*2 + 32*5000*12*12*64*2
bert_intm = 32*5000*768*768*2 + 32*5000*768*2 + 32*5000*768*3072*4
bert_pool = 32*5000*768*4
bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

total =  embedding + bert + 32*8*2*768*768*50

total_isrec, total